# AT2 - Retail dataset preperation for prediction and forecasting

# 1. Loading the dataset

## 1.1. Launch commands to automatically reload modules

In [1]:
%load_ext autoreload
%autoreload 2

## 1.2. Import the packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1.3. Load the training and test dataset into a dataframes called df_train and df_test

In [3]:
df_calendar = pd.read_csv('../data/raw/calendar.csv',low_memory=False)
df_cal_events = pd.read_csv('../data/raw/calendar_events.csv',low_memory=False)
df_week_sell_price = pd.read_csv('../data/raw/items_weekly_sell_prices.csv',low_memory=False)
df_train = pd.read_csv('../data/raw/sales_train.csv',low_memory=False)
df_test = pd.read_csv('../data/raw/sales_test.csv',low_memory=False)

# 2. Exploring the dataset

 ## 2.1 Installing the custom package with pip

In [4]:
! pip install -i https://test.pypi.org/simple/ my_krml_pine==2024.0.1.18

Looking in indexes: https://test.pypi.org/simple/


## 2.2 import custom functions for data exploration

In [5]:
from my_krml_pine.data.dataexplorer import dataexplorer

## 2.3 Data Exploration using custom functions

In [6]:
# Create instances of DataExplorer
explorer = dataexplorer(df_train, df_test)

# Call the methods of the class to perform data exploration tasks on training and test datasets
explorer.display_shape()
explorer.display_columns()
explorer.display_info()
explorer.display_descriptive_stats()
explorer.display_no_of_duplicate_rows()
explorer.calculate_missing_percentage()

Shape of the training dataset:
Rows: 30490, Columns: 1547
Shape of the testing dataset:
Rows: 30490, Columns: 400
--------------------------------------------------------------------------------------------------------------
Columns of the training dataset:
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1532', 'd_1533', 'd_1534', 'd_1535', 'd_1536', 'd_1537', 'd_1538',
       'd_1539', 'd_1540', 'd_1541'],
      dtype='object', length=1547)
--------------------------------------------------------------------------------------------------------------
Columns of the testing dataset:
Index(['d_1542', 'd_1543', 'd_1544', 'd_1545', 'd_1546', 'd_1547', 'd_1548',
       'd_1549', 'd_1550', 'd_1551',
       ...
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938',
       'd_1939', 'd_1940', 'd_1941'],
      dtype='object', length=400)
------------------------------------------------------------

## 2.5 Exploring meta data datasets

In [7]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      1969 non-null   object
 1   wm_yr_wk  1969 non-null   int64 
 2   d         1969 non-null   object
dtypes: int64(1), object(2)
memory usage: 46.3+ KB


In [8]:
df_calendar.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


In [9]:
df_cal_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        167 non-null    object
 1   event_name  167 non-null    object
 2   event_type  167 non-null    object
dtypes: object(3)
memory usage: 4.0+ KB


In [10]:
df_calendar.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


In [11]:
df_week_sell_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [12]:
df_week_sell_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


# 3. Data Preperation

## 3.1 To integrate 'date' from 'calendar' and 'event type' from 'calendar_events' into the training and test datasets 

In [13]:
# 1. Prepare the Date Mapping
# Replace inconsistent date separators (e.g., '-' with '/')
df_calendar['date'] = df_calendar['date'].str.replace('-', '/', regex=False)

# Doing some quick feature engineering from date field before merging
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y/%m/%d',errors='coerce')
df_calendar['day_index'] = df_calendar['d'].str.extract('(\d+)').astype(int)
df_calendar['day_name'] = df_calendar['date'].dt.strftime('%a')
df_calendar['month_name'] = df_calendar['date'].dt.strftime('%b')


In [14]:
# 2. Melting the training data so that 'd_' columns become a single column
melted_train_data = pd.melt(df_train, 
                            id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                            var_name='d', 
                            value_name='sales')

# Now the 'd' column in melted_train_data corresponds to the 'd' column in calendar_data

# 3. Merging with the calendar data
merged_train_data = pd.merge(melted_train_data, df_calendar, on='d', how='left')

In [15]:
merged_train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1


In [16]:
# 4. Merging only the 'event_type' column based on 'date' from the 'calendar_events' meta data

#convert the date column in calendar events dataset
# Replace inconsistent date separators (e.g., '-' with '/')
df_cal_events['date'] = df_cal_events['date'].str.replace('-', '/', regex=False)

df_cal_events['date'] = pd.to_datetime(df_cal_events['date'], format='%Y/%m/%d',errors='coerce')

final_merged_train_data = pd.merge(merged_train_data, df_cal_events[['date', 'event_type']], on='date', how='left')

In [17]:
final_merged_train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month,event_type
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,NaN


In [18]:
# 5. Include the id columns into test dataset
#Extract the first 6 columns from the training dataset (these are the identifier columns)
id_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
train_identifiers = df_train[id_cols]

# Combine the test dataset (which only has d_1542 to d_1941 columns) with the identifiers
# This ensures the test dataset gets the same structure as the training dataset
df_test_with_id = pd.concat([train_identifiers, df_test], axis=1)

# Verify the new structure
print(df_test_with_id.head())

                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1542  d_1543  d_1544  d_1545  ...  d_1932  d_1933  d_1934  \
0       CA       0       1       0       2  ...       2       4       0   
1       CA       0       0       0       0  ...       0       1       2   
2       CA       0       0       0       0  ...       1       0       2   
3       CA       4       1       0       1  ...       1       1       0   
4       CA       3       0       0       1  ...       0       0       0   

   d_1935  d_1936  d_1937  d_1938  d_1939  d_1940  d_1941  
0       

In [19]:
# 6. Melting the test data so that 'd_' columns become a single column
melted_test_data = pd.melt(df_test_with_id, 
                            id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                            var_name='d', 
                            value_name='sales')

# Now the 'd' column in melted_test_data corresponds to the 'd' column in calendar_data

# 7. Merging with the calendar data
merged_test_data = pd.merge(melted_test_data, df_calendar, on='d', how='left')

In [20]:
merged_test_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,4,2015-04-19,11512,1542,Sunday,4
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,3,2015-04-19,11512,1542,Sunday,4


In [21]:
# 8. Merging only the 'event_type' column based on 'date' from the 'calendar_events' meta data

final_merged_test_data = pd.merge(merged_test_data, df_cal_events[['date', 'event_type']], on='date', how='left')

In [22]:
final_merged_test_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month,event_type
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,4,2015-04-19,11512,1542,Sunday,4,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,3,2015-04-19,11512,1542,Sunday,4,NaN


In [23]:
final_merged_train_data.shape

(47107050, 14)

In [24]:
final_merged_test_data.shape

(12196000, 14)

In [25]:
# Display unique values of the 'event_type' column
unique_event_types = final_merged_test_data['event_type'].unique()

# Print the unique values
print(unique_event_types)

[nan 'Cultural' 'National' 'Sporting' 'Religious']


In [26]:
# Display unique values of the 'event_type' column
unique_event_types1 = final_merged_train_data['event_type'].unique()

# Print the unique values
print(unique_event_types1)

[nan 'Sporting' 'Cultural' 'National' 'Religious']


In [27]:
# 9. Replace missing values in event_type feature
final_merged_test_data['event_type'] = final_merged_test_data['event_type'].fillna('No Event')
final_merged_train_data['event_type'] = final_merged_train_data['event_type'].fillna('No Event')

## 3.2 Merge the sales price into training and test datasets and create the target variable 'sales_revenue'

In [28]:
#Merging the final dataset with price meta data
price_merged_test_data = pd.merge(final_merged_test_data, df_week_sell_price, on=['store_id', 'item_id', 'wm_yr_wk'])
price_merged_train_data = pd.merge(final_merged_train_data, df_week_sell_price, on=['store_id', 'item_id', 'wm_yr_wk'])

# Calculate sales revenue
price_merged_test_data['sales_revenue'] = price_merged_test_data['sales'] * price_merged_test_data['sell_price']
price_merged_train_data['sales_revenue'] = price_merged_train_data['sales'] * price_merged_train_data['sell_price']

In [29]:
price_merged_train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month,event_type,sell_price,sales_revenue
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,1,Saturday,1,No Event,0.46,5.52
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,1,Saturday,1,No Event,1.56,3.12
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,No Event,3.17,0.00
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,Saturday,1,No Event,5.98,0.00
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,1,Saturday,1,No Event,0.70,2.80


In [30]:
price_merged_test_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,day_index,day_name,month,event_type,sell_price,sales_revenue
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,No Event,8.26,0.00
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,No Event,3.97,0.00
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,0,2015-04-19,11512,1542,Sunday,4,No Event,2.97,0.00
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,4,2015-04-19,11512,1542,Sunday,4,No Event,4.64,18.56
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1542,3,2015-04-19,11512,1542,Sunday,4,No Event,2.88,8.64


## 3.3 Saving the intermediate results in the data folder

In [31]:
# Training data
price_merged_train_data.to_parquet('../data/interim/price_merged_train_data.parquet', index=False, engine='pyarrow')

# Testing data
price_merged_test_data.to_parquet('../data/interim/price_merged_test_data.parquet', index=False, engine='pyarrow')

## 3.4 Handling missing data 

In [32]:
# Create instances of DataExplorer
explorer = dataexplorer(price_merged_train_data, price_merged_test_data)

# Call the methods of the class to perform data exploration tasks on training and test datasets
explorer.display_shape()
explorer.display_columns()
explorer.display_info()
explorer.display_descriptive_stats()
explorer.display_no_of_duplicate_rows()
explorer.calculate_missing_percentage()

Shape of the training dataset:
Rows: 34815174, Columns: 16
Shape of the testing dataset:
Rows: 12160986, Columns: 16
--------------------------------------------------------------------------------------------------------------
Columns of the training dataset:
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'day_index', 'day_name', 'month',
       'event_type', 'sell_price', 'sales_revenue'],
      dtype='object')
--------------------------------------------------------------------------------------------------------------
Columns of the testing dataset:
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'day_index', 'day_name', 'month',
       'event_type', 'sell_price', 'sales_revenue'],
      dtype='object')
--------------------------------------------------------------------------------------------------------------
Info about training dataset:
<class 'pandas.cor

*** No missing values - So no action was taken